## Import Historical Data from Mongodb

In [1]:
import pandas as pd
from pymongo import MongoClient
from tqdm import tqdm

def fetch_data_in_batches_with_progress(mongo_uri, db_name, collection_name, username, password, batch_size=100000):
    # Format the MongoDB URI with the provided username and password
    mongo_uri = f"mongodb://{username}:{password}@{mongo_uri}"
    client = MongoClient(mongo_uri)
    db = client[db_name]
    collection = db[collection_name]

    # Determine the total number of documents to set up the progress bar
    total_documents = collection.count_documents({})

    # Initialize the progress bar
    pbar = tqdm(total=total_documents, desc='Fetching Data', unit='doc')

    data_batches = []  # Store data in batches
    for skip in range(0, total_documents, batch_size):
        # Use skip and limit to fetch the batch
        cursor = collection.find().skip(skip).limit(batch_size)
        batch = list(cursor)

        # Convert the batch to a DataFrame and add it to the list
        batch_df = pd.DataFrame(batch)
        data_batches.append(batch_df)

        # Update the progress bar
        pbar.update(len(batch))

    # Combine all batches into a single DataFrame
    combined_df = pd.concat(data_batches, ignore_index=True)

    # Close the progress bar and clean up
    pbar.close()
    cursor.close()
    client.close()

    return combined_df

# Use this function to fetch data with progress tracking
data = fetch_data_in_batches_with_progress('mongodb:27017', 'Epsymolo', 'power_flow_all_data', 'root', 'root')

Fetching Data: 100%|██████████| 702/702 [00:01<00:00, 596.19doc/s]


In [2]:
data.dtypes

_id           object
Timestamp     object
0            float64
1            float64
2            float64
              ...   
1911         float64
1912         float64
1913         float64
1914         float64
1915         float64
Length: 1918, dtype: object

In [3]:
df=data

## Data Preprocessing with PySpark

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, hour,expr
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, TimestampType
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lag, abs
from pyspark.ml.feature import VectorAssembler

# Initialize Spark session
spark = SparkSession.builder.master("local").appName("DataProcessing").getOrCreate()

In [ ]:
# Assuming you have a Pandas DataFrame `df` from MongoDB:
df['_id'] = df['_id'].astype(str)

# Now convert the Pandas DataFrame to a PySpark DataFrame
df_spark = spark.createDataFrame(df)

# Drop the '_id' column if it's not needed
df_spark = df_spark.drop("_id")

# Create an array of column names we want to melt, excluding the Timestamp column
value_columns = [c for c in df_spark.columns if c != 'Timestamp']

# Create a list of column expressions for the stack function
stack_exprs = ", ".join(["'{0}', `{0}`".format(c) for c in value_columns])

# Create the stack expression string
stack_expr = "stack({0}, {1}) as (PowerLineID, PowerFlowValue)".format(len(value_columns), stack_exprs)

# Add the Timestamp column back, and use "inline" to convert the structs into rows
df_spark = df_spark.selectExpr("Timestamp", stack_expr)

# Show the first few rows of the melted DataFrame
df_spark.show()
# Convert to Pandas DataFrame for scikit-learn
df = df_spark.toPandas()
# Stop the Spark session
spark.stop()

In [ ]:
# Feature Engineering
windowSpec = Window.partitionBy("PowerLineID").orderBy("Timestamp")
df_spark = df_spark.withColumn("PrevPowerFlowValue", lag("PowerFlowValue").over(windowSpec))
df_spark = df_spark.withColumn("PowerChange", col("PowerFlowValue") - col("PrevPowerFlowValue"))
df_spark = df_spark.na.drop()

# Calculate Threshold
percentile_threshold = 0.75
threshold = df_spark.approxQuantile("PowerChange", [percentile_threshold], 0.05)[0]
df_spark = df_spark.withColumn("IsSignificantChange", (abs(col("PowerChange")) > threshold).cast("integer"))

# Convert to Pandas DataFrame for scikit-learn
df_pandas = df_spark.toPandas()

# Stop the Spark session
spark.stop()

In [ ]:
# Feature Engineering: Calculate the power change
df['PrevPowerFlowValue'] = df.groupby('PowerLineID')['PowerFlowValue'].shift(1)
df['PowerChange'] = df['PowerFlowValue'] - df['PrevPowerFlowValue']

# Drop rows with nulls that were introduced by the shift operation
df = df.dropna()

# Calculate Threshold
percentile_threshold = 0.75
threshold = df['PowerChange'].quantile(percentile_threshold)

# Use the percentile-based threshold to define significant changes
df['IsSignificantChange'] = (df['PowerChange'].abs() > threshold).astype(int)

## Train the model for change detection using Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib

# Assuming df_pandas is your Pandas DataFrame from the previous steps

# Split the data into features and target
X = df[['PowerFlowValue', 'PowerChange']]  # include other features as needed
y = df['IsSignificantChange']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = RandomForestClassifier(n_estimators=10, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print the evaluation metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Confusion Matrix:\n{conf_matrix}")

# Save the model using joblib
joblib.dump(model, 'model.joblib')